In [12]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
meta2017 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2017.csv', encoding = 'utf-8')
meta2018 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2018.csv', encoding = 'utf-8')
meta2019 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2019.csv', encoding = 'utf-8')
meta2020 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2020.csv', encoding = 'utf-8')

#knowcode 결측치 제거
meta2017 = meta2017[meta2017['knowcode'] != 9999999]
meta2018 = meta2018[meta2018['knowcode'] != 9999999]
meta2019 = meta2019[meta2019['knowcode'] != 9999999]
meta2020 = meta2020[meta2020['knowcode'] != 9999999]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (179) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
#밀린거 전처리한 test 데이터 불러오기
metatest2017 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2017_test.csv' ,encoding = 'utf-8')
metatest2018 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2018_test.csv' ,encoding = 'utf-8')
metatest2019 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2019_test.csv' ,encoding = 'utf-8')
metatest2020 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2020_test.csv' ,encoding = 'utf-8')

# 2. 유사도 찾는데 방해될 키워드 제거하기

## 2-1. 키워드 제거함수
- 설정된 제거할 키워드를 컬럼에서 모조리 없에버림

In [16]:
def word_remover(df, word_list, cols):
  for col in cols:
    df[col] = df[col].fillna('')
    for word in word_list:
      if word != '급':
        df[col] = df[col].apply(lambda x: x.replace(word,''))
      else:
        df[col] = df[col].apply(lambda x: x.replace('1급',''))
        df[col] = df[col].apply(lambda x: x.replace('2급',''))
        df[col] = df[col].apply(lambda x: x.replace('3급',''))
        df[col] = df[col].apply(lambda x: x.replace('4급',''))
        df[col] = df[col].apply(lambda x: x.replace('5급',''))
        df[col] = df[col].apply(lambda x: x.replace('6급',''))
        df[col] = df[col].apply(lambda x: x.replace('7급',''))
        df[col] = df[col].apply(lambda x: x.replace('취급',''))

  return df

제거하기 전

In [17]:
meta2017[meta2017['bq4_1a'].str.contains('안경사')]['bq4_1a']

1071          안경사자격증
1253         안경사 면허증
1912             안경사
2055         국가공인안경사
2469          안경사면허증
2886          안경사면허증
4070         안경사 자격증
4168          안경사면허증
4312          안경사면허증
5515             안경사
5521    안경사국가고시자격면허증
6689          안경사면허증
6696          안경사자격증
6742         안경사 면허증
8000          안경사자격증
8537          안경사자격증
Name: bq4_1a, dtype: object

## 2-2. TF-IDF를 적용할 컬럼들과 제거할 키워드 설정

In [18]:
cols = ['bq4_1a','bq4_1b','bq4_1c']
word_list = ['없다', '기사', '기능사', '자격증', '면허증','면허', '토익' ,'정보처리', '학위', '워드프로세서', '워드프로세스',
             '자격', '관련', '일반', '이상','기술사','관리사', '엑셀', '보통', '취득', '전공', '소지자', '면호',
             '국가','공인','급','고시','시험','기술자','산업', '등록', '등록증', '학교', '금',
             '운전','(',')',',',' ']

In [19]:
meta2017 = word_remover(meta2017, word_list, cols)
meta2018 = word_remover(meta2018, word_list, cols)
meta2019 = word_remover(meta2019, word_list, cols)
meta2020 = word_remover(meta2020, word_list, cols)

metatest2017 = word_remover(metatest2017, word_list, cols)
metatest2018 = word_remover(metatest2018, word_list, cols)
metatest2019 = word_remover(metatest2019, word_list, cols)
meta2test020 = word_remover(metatest2020, word_list, cols)

제거 후 -> 이제 안경사끼리는 같은 단어로 파악할 수 있게됨

In [20]:
meta2017[meta2017['bq4_1a'].str.contains('안경사')]['bq4_1a']

1071    안경사
1253    안경사
1912    안경사
2055    안경사
2469    안경사
2886    안경사
4070    안경사
4168    안경사
4312    안경사
5515    안경사
5521    안경사
6689    안경사
6696    안경사
6742    안경사
8000    안경사
8537    안경사
Name: bq4_1a, dtype: object

## 2-3. 제거후 자격증 컬럼끼리 합치기
- 3개로 나누어진 자격증 컬럼을 하나로 합쳐서 유사도 조사를 할 수 있게 한다

In [21]:
meta2017['license'] = meta2017['bq4_1a'] +' '+ meta2017['bq4_1b'] + ' ' + meta2017['bq4_1c']
meta2018['license'] = meta2018['bq4_1a'] +' '+ meta2018['bq4_1b'] + ' ' + meta2018['bq4_1c']
meta2019['license'] = meta2019['bq4_1a'] +' '+ meta2019['bq4_1b'] + ' ' + meta2019['bq4_1c']
meta2020['license'] = meta2020['bq4_1a'] +' '+ meta2020['bq4_1b'] + ' ' + meta2020['bq4_1c']

metatest2017['license'] = metatest2017['bq4_1a'] +' '+ metatest2017['bq4_1b'] + ' ' + metatest2017['bq4_1c']
metatest2018['license'] = metatest2018['bq4_1a'] +' '+ metatest2018['bq4_1b'] + ' ' + metatest2018['bq4_1c']
metatest2019['license'] = metatest2019['bq4_1a'] +' '+ metatest2019['bq4_1b'] + ' ' + metatest2019['bq4_1c']
metatest2020['license'] = metatest2020['bq4_1a'] +' '+ metatest2020['bq4_1b'] + ' ' + metatest2020['bq4_1c']

metatest2017['knowcode'] = -1
metatest2018['knowcode'] = -1
metatest2019['knowcode'] = -1
metatest2020['knowcode'] = -1

# 3. 자격증을 기준으로 유사도를 조사해 KNOWCODE 찾기

<작업방식>

1. 테스트 데이터(Knowcode 없음)에서 자격증 있는 ROW **'한개'**를 트레인 데이터와 합친다
2. 테스트 데이터 'license' 컬럼의 자격증, 자격증들과 유사한 자격증이 있는 Train의 Row들을 TF-IDF기법을 통해 찾아낸다
3. 우리가 정한 유사도 기준 (에: 70%) 이상인 Train의 Row들을 모두 불러낸다
4. 이때 불러온 Train Row들의 Knowcode 들 중에 하나의 Knowcode가 특정 비율 (예: 70%) 이상 이면 1번에서 불러온 테스트 데이터의 Knowcode를 해당 Knowcode로 입력한다

<예시>
1. Test에서 불러온 자격증이 'AAA' 이다
2. Train에서 'AAA'와 유사도가 70% 이상인 자격증들을 찾는다
3. 이때 찾는 것들은 'AAA', 'AAA BBB', 'CCC AAA DDD' 등등이 포함된 Train의 Row 들이 될 수 있다.
4. 그 Train Row의 Knowcode를 보니 '111111' 이 전체에 72%이고, '123456'이 15프로,'234555'가 13프로 이다.
5. 우리의 기준이 70%이니 Test에서 불러온 'AAA'가 있는 Row의 Knowcode를 '111111'로 설정한다

## 3-1. 위에 제시한 방식을 함수로 정리

In [22]:
def tf_idf_license_filter(train_df,test_df,col, sim_score, percent):
  tf_idxs = [] #덮어쓸 인덱스들을 모은 리스트
  tf_knowcode = [] #위 각각 인덱스 마다 덮어 씌울 KNOWCODE

  for idxx in range(len(test_df)): #각 테스트 로우를 학습데이터에 넣어서 학습을 진행할 것이다
    if test_df[col].iloc[idxx] != '  ': #테스트 로우에 자격증이 존재할때만 진행
      one_test_row_and_train_df = pd.concat([test_df.iloc[[idxx]],train_df],axis = 0) #idxx번째 test row를 train데이터 맨위에다가 붙임

      tfidf = TfidfVectorizer(stop_words='english') # stop_words 확인하기
      tfidf_matrix = tfidf.fit_transform(one_test_row_and_train_df[col].values.astype('U')) #위에서 만든 데이터로 학습

      license_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix) #유사한 자격증을 찾은 매트릭스를 만들어냄

      sim_scores = [(i, c) for i, c in enumerate(license_matrix[0]) if i != 0] #테스트 로우의 자격증과 학습데이터의 각 로우에 있는 자격증들과의 유사도
      sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) #유사도가 높은 순으로 정리

      scores = [] #유사도가 특정값 이상인것만 모을 리스트 (인덱스와, 유사도값 모두 저장)
      indexes = [] #유사도가 특정값 이상인 학습데이터의 index를 모을 리스트

      for i in range(len(sim_scores)):
        if sim_scores[i][1] > sim_score: #우리가 지정한 유사도 이상인것만 
          scores.append(sim_scores[i])
          indexes.append(sim_scores[i][0])
        else:
          break

      try:
          high_sim_license_df = one_test_row_and_train_df[[col,'knowcode']].iloc[indexes] #테스트데이터의 자격증과 유사한 자격증을 가진 학습 데이터들을 뽑아 데이터 프레임으로 만듬
          knowcode_idx_count = high_sim_license_df['knowcode'].value_counts() #가장 많이 등장하는 Knowcode를 찾는다
          most_knowcode_counts = knowcode_idx_count.values[0] #가장 많이 등장하는 KNOWCODE의 등장횟수
          knowcode_total_counts = sum(knowcode_idx_count.values) #특정유사도 이상인 모든 학습데이터의 갯수

      

          if most_knowcode_counts/knowcode_total_counts >= percent: #가장많이 등장하는 KNOWCODE가 특정 비율 이상이면
            tf_idxs.append(idxx) #그 테스트데이터의 인덱스는 
            tf_knowcode.append(knowcode_idx_count.index[0]) #추후 그 KNOWCODE로 바꿔줄 것이다
      except:
        pass

  return pd.DataFrame({'idx':tf_idxs, 'knowcode': tf_knowcode})

## 3-2. Test의 자격증을 통해 Train을 조사하여 위의 방식대로 Knowcode들 설정한다

In [23]:
metatest2017[['license']].iloc[341]

license    안경사  
Name: 341, dtype: object

In [24]:
metatest2017[['license']].iloc[1521]

license    기계조립  
Name: 1521, dtype: object

In [25]:
def tf_idf_license_filter_print(train_df,test_df,col, sim_score, percent):
  tf_idxs = [] #덮어쓸 인덱스들을 모은 리스트
  tf_knowcode = [] #위 각각 인덱스 마다 덮어 씌울 KNOWCODE

  for idxx in range(len(test_df)): #각 테스트 로우를 학습데이터에 넣어서 학습을 진행할 것이다
    if test_df[col].iloc[idxx] != '  ': #테스트 로우에 자격증이 존재할때만 진행
      one_test_row_and_train_df = pd.concat([test_df.iloc[[idxx]],train_df],axis = 0) #idxx번째 test row를 train데이터 맨위에다가 붙임

      tfidf = TfidfVectorizer(stop_words='english') # stop_words 확인하기
      tfidf_matrix = tfidf.fit_transform(one_test_row_and_train_df[col].values.astype('U')) #위에서 만든 데이터로 학습

      license_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix) #유사한 자격증을 찾은 매트릭스를 만들어냄

      sim_scores = [(i, c) for i, c in enumerate(license_matrix[0]) if i != 0] #테스트 로우의 자격증과 학습데이터의 각 로우에 있는 자격증들과의 유사도
      sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) #유사도가 높은 순으로 정리

      scores = [] #유사도가 특정값 이상인것만 모을 리스트 (인덱스와, 유사도값 모두 저장)
      indexes = [] #유사도가 특정값 이상인 학습데이터의 index를 모을 리스트

      for i in range(len(sim_scores)):
        if sim_scores[i][1] > sim_score: #우리가 지정한 유사도 이상인것만 
          scores.append(sim_scores[i])
          indexes.append(sim_scores[i][0])
          one_test_row_and_train_df['sim_score'] = sim_scores[i][1]
        else:
          break

      try:

          high_sim_license_df = one_test_row_and_train_df[[col,'knowcode','sim_score']].iloc[indexes] #테스트데이터의 자격증과 유사한 자격증을 가진 학습 데이터들을 뽑아 데이터 프레임으로 만듬
          knowcode_idx_count = high_sim_license_df['knowcode'].value_counts() #가장 많이 등장하는 Knowcode를 찾는다
          most_knowcode_counts = knowcode_idx_count.values[0] #가장 많이 등장하는 KNOWCODE의 등장횟수
          knowcode_total_counts = sum(knowcode_idx_count.values) #특정유사도 이상인 모든 학습데이터의 갯수
          print(high_sim_license_df)
          print()
          print(high_sim_license_df['knowcode'].value_counts())
      except:
        pass

  return pd.DataFrame({'idx':tf_idxs, 'knowcode': tf_knowcode})

In [26]:
x2017 = tf_idf_license_filter_print(meta2017,metatest2017[341:342],'license',sim_score =0.7, percent= 0.7)

     license  knowcode  sim_score
1071   안경사      307301        1.0
1253   안경사      307301        1.0
1912   안경사      307301        1.0
2055   안경사      307301        1.0
2469   안경사      307301        1.0
2886   안경사      307301        1.0
4070   안경사      307301        1.0
4168   안경사      307301        1.0
4312   안경사      307301        1.0
5515   안경사      307301        1.0
5521   안경사      307301        1.0
6689   안경사      307301        1.0
6696   안경사      307301        1.0
6742   안경사      307301        1.0
8000   안경사      307301        1.0
8537   안경사      307301        1.0

307301    16
Name: knowcode, dtype: int64


In [27]:
x2017 = tf_idf_license_filter_print(meta2017,metatest2017[1521:1522],'license',sim_score =0.7, percent= 0.7)

     license  knowcode  sim_score
973   기계조립      817201        1.0
1508  기계조립      817201        1.0
2514  기계조립      817201        1.0
4142  기계조립      817201        1.0
4809  기계조립      816103        1.0
5151  기계조립      816103        1.0
6129  기계조립      817101        1.0
6258  기계조립      214301        1.0
6398  기계조립      817101        1.0
6458  기계조립      214301        1.0
8992  기계조립      214301        1.0

817201    4
214301    3
817101    2
816103    2
Name: knowcode, dtype: int64


In [28]:
x2017 = tf_idf_license_filter(meta2017,metatest2017,'license',sim_score =0.7, percent= 0.67)
x2018 = tf_idf_license_filter(meta2018,metatest2018,'license',sim_score =0.7, percent= 0.67)
x2019 = tf_idf_license_filter(meta2019,metatest2019,'license',sim_score =0.7, percent= 0.67)
x2020 = tf_idf_license_filter(meta2020,metatest2020,'license',sim_score =0.7, percent= 0.67)

KeyboardInterrupt: ignored

제출용 파일에 맞게끔 인덱스를 정리해주는 작업 (위에서 함수를 제대로 못만들어 하는 과정)

In [ ]:
for i in range(len(x2017)):
  x2017['idx'].iloc[i] = metatest2017['idx'].iloc[x2017['idx'][i]]
for i in range(len(x2018)):
  x2018['idx'].iloc[i] = metatest2018['idx'].iloc[x2018['idx'][i]]
for i in range(len(x2019)):
  x2019['idx'].iloc[i] = metatest2019['idx'].iloc[x2019['idx'][i]]
for i in range(len(x2020)):
  x2020['idx'].iloc[i] = metatest2020['idx'].iloc[x2020['idx'][i]]

연도별로 찾은 자격증별 Knowcode를 합쳐줌

In [ ]:
idx_knowcode_df = pd.concat([x2017,x2018],axis = 0)
idx_knowcode_df = pd.concat([idx_knowcode_df,x2019],axis = 0)
idx_knowcode_df = pd.concat([idx_knowcode_df,x2020],axis = 0)
idx_knowcode_df #추후 submission 데이터와 용이하게 합치기위해 합쳐줌

In [ ]:
idx_knowcode_df.to_csv('/gdrive/MyDrive/2조/data/코드정리/TF-IDF로 찾은 덮어쓰기/최소유사도 0.7이상이고 전체에 68프로가 해당되는 Knowcode 덮어쓰기.csv',index = False)

## 3-3 기존 학습된 파일에다가 합쳐주기
- Voting Classifer를 통해 얻어낸 테스트 결과물에
- 위에서 찾은것을 덮어씌워준다

모델 학습으로 만든 파일 불러오기

In [ ]:
sub = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub_1차.csv')
sub

TF-IDF 기법으로 찾은 것과 합쳐주기

In [ ]:
new_sub = pd.merge(sub, idx_knowcode_df, how = 'left',on = 'idx')
new_sub.columns = ['idx','knowcode','newcode']
new_sub = new_sub.fillna(0)
new_sub

In [ ]:
new_sub['differentcode'] = 0

In [ ]:
#newcode에서 변환하기위해 준비된 값들을 제출용 knowcode로 덮어쓰기함
for i in range(len(new_sub)):
  if new_sub['newcode'].iloc[i] !=  0:
    new_sub['knowcode'].iloc[i] = int(new_sub['newcode'].iloc[i])

new_sub

In [ ]:
sub = new_sub[['idx','knowcode']]
sub

저장

In [ ]:
sub.to_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub_1차에 최소유사도 0.7이상이고 전체에 70프로가 해당되는 Knowcode 덮어쓰기.csv',index = False)

# 4. 자격증을 **다른기준** 유사도를 조사해 KNOWCODE 찾기 - 

<작업방식>

1. 테스트 데이터(Knowcode 없음)에서 자격증 있는 ROW **'한개'**를 트레인 데이터와 합친다
2. 테스트 데이터 'license' 컬럼의 자격증, 자격증들과 유사한 자격증이 있는 Train의 Row들을 TF-IDF기법을 통해 찾아낸다
3. 우리가 정한 유사도 기준 (에: 40%) 이상인 Train의 Row들을 모두 불러낸다
4. **다만 유사도 중 최고치로 부터 30% 이하로 해당되는것만 다음단계에서 진행한다**   (최고 유사도가 90프로 이면 유사도가 60프로 이상인 것들까지만 쓰인다. 40프로는 3번에서 걸러지지만 5번에서 쓰이지 않음)
5. 이때 불러온 Train Row들의 Knowcode 들 중에 하나의 Knowcode가 특정 비율 (예: 70%) 이상 이면 1번에서 불러온 테스트 데이터의 Knowcode를 해당 Knowcode로 입력한다

<예시>
1. Test에서 불러온 자격증이 'AAA' 이다
2. Train에서 'AAA'와 유사도가 70% 이상인 자격증들을 찾는다
3. 이때 찾는 것들은 'AAA', 'AAA BBB', 'CCC AAA DDD', 'JJJJ KKKK AAA'등등이 포함된 Train의 Row 들이 될 수 있다.
4. 다만 'AAA'가 'AAA'와 유사도가 100프로인데 'JJJJ KKKK AAA'는 유사도가 70프로보다 낮아 해당되지 않는다
4. 그 Train Row의 Knowcode를 보니 '111111' 이 전체에 72%이고, '123456'이 15프로,'234555'가 13프로 이다.
5. 우리의 기준이 70%이니 Test에서 불러온 'AAA'가 있는 Row의 Knowcode를 '111111'로 설정한다

## 4-1. 위에 제시한 방식을 함수로 정리

In [29]:
def tf_idf_license_filter2(train_df,test_df,col, sim_score_from_max_sim, min_sim_score, percent):
  tf_idxs = [] #덮어쓸 인덱스들을 모은 리스트
  tf_knowcode = [] #위 각각 인덱스 마다 덮어 씌울 KNOWCODE

  for idxx in range(len(test_df)): #각 테스트 로우를 학습데이터에 넣어서 학습을 진행할 것이다
    if test_df[col].iloc[idxx] != '  ': #테스트 로우에 자격증이 존재할때만 진행
      one_test_row_and_train_df = pd.concat([test_df.iloc[[idxx]],train_df],axis = 0) #idxx번째 test row를 train데이터 맨위에다가 붙임

      tfidf = TfidfVectorizer(stop_words='english') # stop_words 확인하기
      tfidf_matrix = tfidf.fit_transform(one_test_row_and_train_df[col].values.astype('U')) #위에서 만든 데이터로 학습

      license_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix) #유사한 자격증을 찾은 매트릭스를 만들어냄

      sim_scores = [(i, c) for i, c in enumerate(license_matrix[0]) if i != 0 and c >min_sim_score] #테스트 로우의 자격증과 학습데이터의 각 로우에 있는 자격증들과의 유사도 #지정한 최소유사도만 들어갈수 있다
      sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) #유사도가 높은 순으로 정리

      scores = [] #유사도가 특정값 이상인것만 모을 리스트 (인덱스와, 유사도값 모두 저장)
      indexes = [] #유사도가 특정값 이상인 학습데이터의 index를 모을 리스트

      if len(sim_scores) > 0:
        max_sim_score = sim_scores[0][1] #유사도중 가장 큰 값
        for i in range(len(sim_scores)):
          if sim_scores[i][1] > max_sim_score - sim_score_from_max_sim: #최대 유사도에서 0.3(지정한값) 아래 사이에 있는것만 들어갈수 있다
            scores.append(sim_scores[i])
            indexes.append(sim_scores[i][0])
            
          else:
            break

      try:
          high_sim_license_df = one_test_row_and_train_df[[col,'knowcode']].iloc[indexes] #테스트데이터의 자격증과 유사한 자격증을 가진 학습 데이터들을 뽑아 데이터 프레임으로 만듬
          knowcode_idx_count = high_sim_license_df['knowcode'].value_counts() #가장 많이 등장하는 Knowcode를 찾는다
          most_knowcode_counts = knowcode_idx_count.values[0] #가장 많이 등장하는 KNOWCODE의 등장횟수
          knowcode_total_counts = sum(knowcode_idx_count.values) #특정유사도 이상인 모든 학습데이터의 갯수

      

          if most_knowcode_counts/knowcode_total_counts >= percent: #가장많이 등장하는 KNOWCODE가 특정 비율 이상이면
            tf_idxs.append(idxx) #그 테스트데이터의 인덱스는 
            tf_knowcode.append(knowcode_idx_count.index[0]) #추후 그 KNOWCODE로 바꿔줄 것이다
      except:
        pass

  return pd.DataFrame({'idx':tf_idxs, 'knowcode': tf_knowcode})

    

## 4-2. Test의 자격증을 통해 Train을 조사하여 위의 방식대로 Knowcode들 설정한다

In [32]:
x2017 = tf_idf_license_filter2(meta2017,metatest2017,'license',sim_score_from_max_sim = 0.5, min_sim_score =0.2, percent= 0.7)
x2018 = tf_idf_license_filter2(meta2018,metatest2018,'license',sim_score_from_max_sim = 0.5, min_sim_score =0.2, percent= 0.7)
x2019 = tf_idf_license_filter2(meta2019,metatest2019,'license',sim_score_from_max_sim = 0.5, min_sim_score =0.2, percent= 0.7)
x2020 = tf_idf_license_filter2(meta2020,metatest2020,'license',sim_score_from_max_sim = 0.5, min_sim_score =0.2, percent= 0.7)

제출용 파일에 맞게끔 인덱스를 정리해주는 작업 (위에서 함수를 제대로 못만들어 하는 과정)

In [33]:
for i in range(len(x2017)):
  x2017['idx'].iloc[i] = metatest2017['idx'].iloc[x2017['idx'][i]]
for i in range(len(x2018)):
  x2018['idx'].iloc[i] = metatest2018['idx'].iloc[x2018['idx'][i]]
for i in range(len(x2019)):
  x2019['idx'].iloc[i] = metatest2019['idx'].iloc[x2019['idx'][i]]
for i in range(len(x2020)):
  x2020['idx'].iloc[i] = metatest2020['idx'].iloc[x2020['idx'][i]]

연도별로 찾은 자격증별 Knowcode를 합쳐줌

In [34]:
idx_knowcode_df = pd.concat([x2017,x2018],axis = 0)
idx_knowcode_df = pd.concat([idx_knowcode_df,x2019],axis = 0)
idx_knowcode_df = pd.concat([idx_knowcode_df,x2020],axis = 0)
idx_knowcode_df #추후 submission 데이터와 용이하게 합치기위해 합쳐줌

,idx,knowcode
0,6,232101
1,14,301004
2,20,541301
3,26,550201
4,32,301004
...,...,...
1660,35231,811201
1661,35235,702201
1662,35240,155303
1663,35241,29401


In [35]:
idx_knowcode_df.to_csv('/gdrive/MyDrive/2조/data/코드정리/TF-IDF로 찾은 덮어쓰기/최소유사도 0.2이상이고 최대로부터 0.5아래 중 전체에 70프로가 해당되는 Knowcode 덮어쓰기.csv',index = False)

## 4-3 기존 학습된 파일에다가 합쳐주기
- Voting Classifer를 통해 얻어낸 테스트 결과물에
- 위에서 찾은것을 덮어씌워준다

모델 학습으로 만든 파일 불러오기

In [36]:
sub = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub54_VC.csv')
sub

,idx,knowcode
0,0,29401
1,1,415101
2,2,412001
3,3,121102
4,4,412003
...,...,...
35226,35244,811101
35227,35245,29303
35228,35246,833001
35229,35247,833001


TF-IDF 기법으로 찾은 것과 합쳐주기

In [37]:
new_sub = pd.merge(sub, idx_knowcode_df, how = 'left',on = 'idx')
new_sub.columns = ['idx','knowcode','newcode']
new_sub = new_sub.fillna(0)
new_sub

,idx,knowcode,newcode
0,0,29401,0.0
1,1,415101,0.0
2,2,412001,0.0
3,3,121102,0.0
4,4,412003,0.0
...,...,...,...
35226,35244,811101,833001.0
35227,35245,29303,0.0
35228,35246,833001,0.0
35229,35247,833001,0.0


In [38]:
new_sub['differentcode'] = 0

In [39]:
#newcode에서 변환하기위해 준비된 값들을 제출용 knowcode로 덮어쓰기함
for i in range(len(new_sub)):
  if new_sub['newcode'].iloc[i] !=  0:
    new_sub['knowcode'].iloc[i] = int(new_sub['newcode'].iloc[i])

new_sub

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,idx,knowcode,newcode,differentcode
0,0,29401,0.0,0
1,1,415101,0.0,0
2,2,412001,0.0,0
3,3,121102,0.0,0
4,4,412003,0.0,0
...,...,...,...,...
35226,35244,833001,833001.0,0
35227,35245,29303,0.0,0
35228,35246,833001,0.0,0
35229,35247,833001,0.0,0


In [40]:
sub = new_sub[['idx','knowcode']]
sub

,idx,knowcode
0,0,29401
1,1,415101
2,2,412001
3,3,121102
4,4,412003
...,...,...
35226,35244,833001
35227,35245,29303
35228,35246,833001
35229,35247,833001


저장

In [41]:
sub.to_csv('/gdrive/MyDrive/2조/data/코드정리/Submission/sub54_VC에 최소유사도 0.2이상이고 최대로부터 0.5아래 중 전체에 70프로가 해당되는 Knowcode 덮어쓰기.csv',index = False)

In [41]:
2